In [5]:
from collections import Counter
from sage.combinat.partition import Partitions
from sage.arith.misc import factor

# Theorem 1 from "On the enumeration of tanglegrams and tangled chains - Sara Billey, Matjaž Konvalinka, Frederick A Matsen IV"

def z_lambda(lam):
    count = Counter(lam)
    z = 1
    for part, mult in count.items():
        z *= part^mult * factorial(mult)
    return z

def is_binary_partition(lam):
    for part in lam:
        if part == 0:
            return False
        if part & (part - 1) != 0:
            return False
    return True

def tanglegram_count(n):
    total = 0
    for lam in Partitions(n):
        if not is_binary_partition(lam):
            continue

        ell = len(lam)
        
        numerator = 1
        for i in range(2, ell + 1): 
            partial_sum = sum(lam[i-1:]) 
            numerator *= (2 * partial_sum - 1)^2
            
        total += numerator / z_lambda(lam)
    
    return total

for n in range(1, 8):
    t_n = tanglegram_count(n)
    print(f"t_{n} = {t_n}")

t_1 = 1
t_2 = 1
t_3 = 2
t_4 = 13
t_5 = 114
t_6 = 1509
t_7 = 25595


In [14]:
import itertools
from collections import defaultdict
from itertools import chain

WE = defaultdict(lambda: None)
WE[0] = 0 
WE[1] = 1

#outputs the Wedderburn number for n
def W(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    if WE[n] == None: 
        if n % 2 == 1:
            s = sum([W(i) * W(n-i) for i in range(1, (n+1)//2)])
            WE[n] = s
        else:
            s = sum([W(i) * W(n-i) for i in range(1, n//2)])
            s += (W(n//2) * (W(n/2) + 1)) / 2  #W(n/2) choose 2 + W(n/2)
            WE[n] = s
    return WE[n]

#in: L list, unordered with duplicates
#out: ordered list L without duplicates
def Oset(L):
    return sorted(list(dict.fromkeys(L)))

#inputs two list of pairs of integers lst1 and lst2 
#outputs a list of ordered pairs ((x,y),(z,w)) 
#such that (x,y) <= (z,w), and (x,y) in lst1 and (z,w) in lst2 or vice versa
def unordered_pairs(lst1, lst2):
    if len(lst1) == 0 or len(lst1) == 0:
        return []
    else: 
        L = [(y, x) if x <= y else (x, y) for x, y in set(itertools.product(lst1, lst2))]
        return Oset(L)

#inputs tree_0 = (n_0,s_0) and tree_1 = (n_1,s_1)
#outputs tree = (n,s) such that tree is tree_0 \oplus tree_1
def binary_tree(tree_0,tree_1):
    ((n_0,s_0),(n_1,s_1)) = sorted((tree_0,tree_1))
    if n_0 == 0: 
        return (n_1,s_1)
    if n_1 == 0: 
        return (n_0,s_0)
    n = n_0 + n_1
    if n_0 == n_1:
        y = W(n_0)
        s = sum([ W(n - i) * W(i) for i in range(1,n_0)]) + sum([ y-i for i in range(s_0)]) + s_1 - s_0
    else: 
        s = sum([ W(n - i) * W(i) for i in range(1,n_0)]) +  W(n_0) * s_1 + s_0
    return (n,s)

#in: a binary tree in the form pair = (n,s)  
#out: draws it in ascii_art  (using the BinaryTree class of Sagemath)
def draw(pair):
    def convert_back(pair):
        T_1 = BinaryTree([])
        (n,s) = (pair[0],pair[1])
        if n == 1:
            return T_1
        if n > 1:
            pair = root_split_finder((n,s))
            T_left = convert_back(pair[0])
            T_right = convert_back(pair[1])
            return BinaryTree([T_left,T_right])
    print(ascii_art(convert_back(pair)))    

root_split_cache = {}

# input: pairs of integers (n,t)
# output: the root split of s'th binary tree (in lex order) on n leaves in the form ((n_0,s_0),(n_1,s_1))
def root_split_finder(pair):
    if pair in root_split_cache:
        return root_split_cache[pair]
    n, t = pair
    if n == 1:
        return ((1,0),(0,0))
    elif n % 2 == 1:
        i = 1
        while W(n-i) * W(i) <= t: 
            t-=W(n-i) * W(i)
            i += 1
        (s_1,s_2) = divmod(int(t),int(W(i)))
        return ((n-i,s_1),(i,s_2))
    else:
        i = 1
        while i <= n/2 and W(n-i) * W(i) <= t: 
            t-=W(n-i) * W(i)
            i+=1      
        (s_1,s_2) = divmod(int(t),int(W(i)))
        if i != n/2:
            return ((n-i,s_1),(i,s_2))
        else:
            y = W(n//2)
            j = 0
            r = 0
            while y-j-1 < t:
                t -= (y-j)
                j += 1
                r += 1
            return ((n//2,t+r),(n//2,j))     

m=defaultdict(lambda: defaultdict(lambda: None ))


#in: a binary tree in the form pair = (n,s)  
#out: draws it in ascii_art  (using the BinaryTree class of Sagemath)
def draw(pair):
    def convert_back(pair):
        T_1 = BinaryTree([])
        (n,s) = (pair[0],pair[1])
        if n == 1:
            return T_1
        if n > 1:
            pair = root_split_finder((n,s))
            T_left = convert_back(pair[0])
            T_right = convert_back(pair[1])
            return BinaryTree([T_left,T_right])
    print(ascii_art(convert_back(pair)))    

#in: a tree = (n,s) 
#out: the tree (n+1,t) = (n,s) \oplus (1,0)
def add_one(T):
    return binary_tree(T,(1,0))

#in: a tree = (n,s) 
#out: the tree (n+2,t) = (n,s) \oplus (2,0)
def add_two(T):
    return binary_tree(T,(2,0))

#in: a tree = (n,s) 
#out: the tree (n+3,t) = (n,s) \oplus (3,0)
def add_three(T): 
    return binary_tree(T,(3,0))

#in: permutation and dictionary of left and right buttons in the form {0: [], 1:[]}
#out: pressed permutation
def press_buttons(p,dct):
    
    #in: permutation and list of left buttons
    def press_left_buttons(p,buttons):
        if not buttons:
            return p
        else:
            button = buttons[0]
            if not button:
                return p
            else:
                midpoint = mid(button)
                q = Permutation(p[ : button[0]-1] + p[ midpoint  : button[1] ]+p[ button[0]-1 : midpoint] + p[ button[1] : ])
                return press_left_buttons(q,buttons[1:])
            
    #in: permutation and list of left buttons
    def press_right_buttons(p,buttons):
        if not buttons:
            return p
        else:
            button = buttons[0]
            if not button:
                return p
            else:
                q = p.inverse()
                midpoint = mid(button)
                q_pressed = Permutation(q[ : button[0]-1] + q[ midpoint  : button[1] ]+q[ button[0]-1 : midpoint] + q[ button[1] : ])
                return press_right_buttons(q_pressed.inverse(),buttons[1:])
            
    q = press_left_buttons(p,dct[0])
    return press_right_buttons(q,dct[1])


#in: permutation p and a button as a pair
#out: 
def mid(button):
    (a,b) = (button[0],button[1])
    if a == b:
        return a-1
    else:
        return (a + b -1)//2

#in: button
#out: length of button
def length_of(button):
    return (button[1] - button[0] + 1)/2

#in: a pattern pat on the elements 1,...,i-1,i+1,...,n
#out: the permutation p on the elements 1,...,n-1
def perm_canonicalizer(pat,i):
    return  Permutation([x+1 if x<i else x for x in pat]  )

#input: a tanglegrams ((n,s_0),(n,s_1),p)
#output: the canonical tanglegram ((m,t_0),(m,t_1),r) in the isom. class of the given tanglegram
def tan_canonicalize(tan):
    ltree = tan[0]
    rtree = tan[1]
    p = tan[2]
    n = ltree[0]
    
    left_bus = set()
    right_bus = set()

    left_bu_list = defaultdict(lambda: [])
    right_bu_list = defaultdict(lambda: [])
    
    # input: pair of integers (n,s) denoting the tree and pair of integers (a,b) denoting the leaf number interval
    # output: the root split of s'th binary tree (in lex order) on n leaves in the form ((n_0,s_0),(n_1,s_1))
    def root_split_finder_with_left_buttons(pair, leaf_interval):
        (ltree, rtree) = root_split_finder(pair)
        if ltree ==rtree:
            left_bus.add(leaf_interval)
        if ltree[0] > 1:
            lsub_leaf_interval = (leaf_interval[0],leaf_interval[0] + ltree[0] -1)
            root_split_finder_with_left_buttons(ltree,lsub_leaf_interval)
        if rtree[0] > 1:
            rsub_leaf_interval = (leaf_interval[0] + ltree[0], leaf_interval[1])
            root_split_finder_with_left_buttons(rtree,rsub_leaf_interval)

    # input: pair of integers (n,s) denoting the tree and pair of integers (a,b) denoting the leaf number interval
    # output: the root split of s'th binary tree (in lex order) on n leaves in the form ((n_0,s_0),(n_1,s_1))
    def root_split_finder_with_right_buttons(pair,leaf_interval):
        (ltree, rtree) = root_split_finder(pair)
        if ltree == rtree:
            right_bus.add(leaf_interval)
        if ltree[0] > 1:
            lsub_leaf_interval = (leaf_interval[0],leaf_interval[0] + ltree[0] -1)
            root_split_finder_with_right_buttons(ltree,lsub_leaf_interval)
        if rtree[0] > 1:
            rsub_leaf_interval = (leaf_interval[0] + ltree[0],leaf_interval[1])
            root_split_finder_with_right_buttons(rtree,rsub_leaf_interval)

    root_split_finder_with_left_buttons(ltree,(1,n))
    root_split_finder_with_right_buttons(rtree,(1,n))

    left_pressing_seq = {}
    right_pressing_seq = {}
    
    left_triples = {}
    right_triples = {}
    
    left_buttons = deepcopy(left_bus)
    right_buttons = deepcopy(right_bus)    
    
    left_button_list = {}
    right_button_list = {}
    
    transformed = {deepcopy(p)}
    min_values = {deepcopy(p)}
    
    # contains a list of dictionaries containing left and right buttons to press, which ultimately yield the canonical form

    for i in range(1,n):
        left_bu_list[i] = [button for button in left_buttons if button[0] <= i < button[0]+length_of(button)]
        right_bu_list[i] = [button for button in right_buttons if button[0] <= i < button[0]+length_of(button)]
    
    for i in range(1,n):
        for j in range(1,n+1):
            left_button_list[j] = [button for button in left_buttons if button[0] <= j < button[0]+length_of(button)]
            right_button_list[j] = [button for button in right_buttons if button[0] <= j < button[0]+length_of(button)]
            left_triples[j] = [(j,button,j+length_of(button)) for button in left_button_list[j]]
            right_triples[j] = [(j,button,j+length_of(button)) for button in right_button_list[j]]

        all_left_triples = [elem for sublist in left_triples.values() for elem in sublist]
        all_right_triples = [elem for sublist in right_triples.values() for elem in sublist]  

        for j in range(1,n+1):
            left_pressing_seq[j] = [[(j,[],j)]]+[seq for seq in Combinations(all_left_triples) if seq != [] and seq[0][0] == j and all(seq[i-1][2] == seq[i][0] for i in range(1,len(seq)))]
            right_pressing_seq[j] = [[(j,[],j)]]+[seq for seq in Combinations(all_right_triples) if seq != [] and seq[-1][-1] == j and all(seq[i-1][2] == seq[i][0] for i in range(1,len(seq)))]
        
        for r in min_values:
            for seq_left in left_pressing_seq[i]:
                im = r[seq_left[-1][-1]-1]
                for seq_right in right_pressing_seq[im]:
                    dct = { 0: list( reversed([ triple[1] for triple in seq_left] )) , 1: list( reversed([ triple[1] for triple in seq_right ])) }
                    q = press_buttons(r,dct)
                    if q[:i] == min(min_values)[:i]:
                        transformed.add(q)
                    if q[:i] < min(min_values)[:i]:
                        transformed = {q}
                    min_so_far = min([elem[:i] for elem in transformed])
                    min_values = set([elem for elem in transformed if elem[:i] == min_so_far ])
        left_buttons = set([button for button in left_buttons if button not in left_bu_list[i]])
        right_buttons = set([button for button in right_buttons if button not in right_bu_list[min_so_far[i-1]]])

    return (ltree, rtree, Permutation(min(min_values)))

#input: a tanglegram T as a tuple ((n,s),(n,t),sigma)
#output: the multideck of cards of T induced by (n-1)-subsets of the n matching edges 
def tan_multideck(tan):
    cbus = defaultdict(lambda: {0: [], 1:[]})
    #input:  pair = (n,s) and an (n-1)-tuple tup of increasing integers i, with 0\leq i \leq n-1, and a fixed integer j
    #output: the card of (n,s), induced by removing leaf j, as an ordered set of pairs
    def card_finder_with_left_buttons(pair, tup, j):
        n = pair[0]
        s = pair[1]
        k = len(tup)
        if k == 0: 
            card = (0,0)
        if k == n:
            card = pair
        if n > k > 0:
            root_split = root_split_finder(pair)
            left_tree = root_split[0]
            right_tree = root_split[1]
            left_sublist = tuple([ i for i in tup if i < left_tree[0] ])
            right_sublist = tuple([ i for i in tup if left_tree[0] <= i])
            right_sublist_adjusted = tuple([ i - left_tree[0] for i in right_sublist])
            left_card = card_finder_with_left_buttons( left_tree, left_sublist, j)
            right_card = card_finder_with_left_buttons( right_tree, right_sublist_adjusted, j)
            card = binary_tree(left_card,right_card)
            if left_card < right_card and 1 < left_tree[0] and k > 2:
                tup_canon = perm_canonicalizer(tup,j)
                cbutton = (tup_canon[0],tup_canon[-1])
                cbus[j][0].append(cbutton)
        return card
    
    def card_finder_with_right_buttons(pair, tup, j, p):
        n = pair[0]
        s = pair[1]
        k = len(tup)
        if k == 0:
            card = (0,0)
        if k == n:
            card = pair
        if n > k > 0:
            root_split = root_split_finder(pair)
            left_tree = root_split[0]
            right_tree = root_split[1]
            left_sublist = tuple([ i for i in tup if i < left_tree[0] ])
            right_sublist = tuple([ i for i in tup if left_tree[0] <= i])
            right_sublist_adjusted = tuple([ i - left_tree[0] for i in right_sublist])
            left_card = card_finder_with_right_buttons( left_tree, left_sublist, j, p)
            right_card = card_finder_with_right_buttons( right_tree, right_sublist_adjusted, j, p)
            card = binary_tree(left_card,right_card)
            if left_card < right_card and 1 < left_tree[0] and k > 2:
                tup_canon = perm_canonicalizer(tup,j)
                cbutton = (tup_canon[0],tup_canon[-1])
                a = p[j-1]
                cbus[a][1].append(cbutton)
        return card
    
    to_return = defaultdict(lambda: 0)
    ltree = tan[0]
    rtree = tan[1]
    p = tan[2]
    n = ltree[0]
    for j in range(1,n+1):
        i = p[j-1]
        subset_left = tuple(range(j-1)) + tuple(range(j,n))
        subset_right = tuple([p[k]-1 for k in range(n) if  k != j-1 ])
        sorted_subset_right = tuple(range(i-1)) + tuple(range(i,n))
        card_ltree = card_finder_with_left_buttons(ltree, subset_left, j)
        card_rtree = card_finder_with_right_buttons(rtree, sorted_subset_right, i, p.inverse())
        card = (card_ltree, card_rtree, perm_canonicalizer(subset_right,i-1) )
        card_pressed = (card_ltree, card_rtree, press_buttons(card[2],cbus[j]) )
        to_return[ tan_canonicalize ( card_pressed ) ]+=1
    return dict(to_return)

#in: tanglegram tan
#out: deck of tan as the keys of a dictionary
def tan_deck(tan):
    return tan_multideck(tan).keys()


In [10]:
#Small example for n = 4
tan_1 = ((4, 0), (4, 0), Permutation([1, 3, 4, 2])) 
tan_2 = ((4, 0), (4, 0), Permutation([1, 4, 2, 3]))
multideck_1 = tan_multideck(tan_1)
multideck_2 = tan_multideck(tan_2)
print("Same multideck:", multideck_1 == multideck_2 )

toprint = '\n'.join(str(x) + " with multiplicity " + str(multideck_1[x]) for x in multideck_1)
print(toprint)
print("Isomorphic tanglegrams:", tan_canonicalize(tan_1)  == tan_canonicalize(tan_2) )
print(tan_canonicalize(tan_1))
print(tan_canonicalize(tan_2))

Same multideck: True
((3, 0), (3, 0), [1, 3, 2]) with multiplicity 3
((3, 0), (3, 0), [1, 2, 3]) with multiplicity 1
Isomorphic tanglegrams: False
((4, 0), (4, 0), [1, 3, 4, 2])
((4, 0), (4, 0), [1, 4, 2, 3])


In [11]:
#Small example for n = 5
tan_1 = ((5, 0), (5, 0), Permutation([3, 5, 4, 1, 2]))
tan_2 = ((5, 0), (5, 0), Permutation([4, 5, 1, 3, 2]))
multideck_1 = tan_multideck(tan_1)
multideck_2 = tan_multideck(tan_2)
print("Same multideck:", multideck_1 == multideck_2)

toprint = '\n'.join(str(x) + " with multiplicity " + str(multideck_1[x]) for x in multideck_1)
print(toprint)
print("Isomorphic tanglegrams:", tan_canonicalize(tan_1)  == tan_canonicalize(tan_2) )
print(tan_canonicalize(tan_1))
print(tan_canonicalize(tan_2))

Same multideck: True
((4, 0), (4, 0), [3, 4, 1, 2]) with multiplicity 3
((4, 0), (4, 0), [1, 4, 3, 2]) with multiplicity 2
Isomorphic tanglegrams: False
((5, 0), (5, 0), [3, 5, 4, 1, 2])
((5, 0), (5, 0), [4, 5, 1, 3, 2])


In [17]:
# Verify isomorphism test
n = 5
k = W(n)
list_tan = set()
counter = 0
for bl in range(k):
    for br in range(k):
        for perm in Permutations(n):
            counter += 1
            list_tan.add(tan_canonicalize(((n,bl),(n,br),perm)))
            
# Exhaustive Search for counterexamples when n = 5
n = 5
k = W(n) 
Counterexamples = []
for tan_1 in list_tan:
    for tan_2 in list_tan:
        multideck_1 = tan_multideck(tan_1)
        multideck_2 = tan_multideck(tan_2)
        if multideck_1 == multideck_2 and tan_1 != tan_2:  
            Counterexamples.append( [ tan_1, tan_2 ] )
            print("-"*20)
            print(tan_1,tan_2)
            if tan_1[2].inverse() != tan_2[2]: 
                print("no switching")
            else:
                print("switch left right trees")

print(len(Counterexamples))

--------------------
((5, 0), (5, 0), [3, 5, 4, 1, 2]) ((5, 0), (5, 0), [4, 5, 1, 3, 2])
switch left right trees
--------------------
((5, 1), (5, 0), [1, 5, 3, 4, 2]) ((5, 1), (5, 0), [1, 4, 3, 5, 2])
no switching
--------------------
((5, 0), (5, 0), [4, 5, 1, 3, 2]) ((5, 0), (5, 0), [3, 5, 4, 1, 2])
switch left right trees
--------------------
((5, 0), (5, 1), [1, 5, 3, 2, 4]) ((5, 0), (5, 1), [1, 5, 3, 4, 2])
no switching
--------------------
((5, 0), (5, 1), [1, 5, 3, 4, 2]) ((5, 0), (5, 1), [1, 5, 3, 2, 4])
no switching
--------------------
((5, 1), (5, 0), [1, 4, 3, 5, 2]) ((5, 1), (5, 0), [1, 5, 3, 4, 2])
no switching
6
